### PyTorch-SENet

**SENet** was winner of 2017 ImageNet classification challenge. In this network architecture authors introduced a new block called *Squeeze and Excitation* block. In a vanilla CNN, parameters of each channel of a convolutional block are simply added together, main idea of a **SEBlock** is to explicitly model the weights of each feature map. In the notebook below we will experiment with **SENet** and take a look at the **SEBlock**.

https://arxiv.org/pdf/1709.01507.pdf

In [2]:
#Perform imports
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid

import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
#Import models
from torchvision import models

In [7]:
#Lets use torchhub to download SENet
import torch.hub
senet = torch.hub.load(
    'moskomule/senet.pytorch',
    'se_resnet50',
    pretrained=True,)

Using cache found in /Users/jamieott/.cache/torch/hub/moskomule_senet.pytorch_master


In [8]:
senet

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): SEBottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (se): SELayer(
        (avg_pool): AdaptiveAvgPool2d(output_size=1)
        (fc): Se

We can see the **SELayer** in the model, these layers take average pooled output of the conv layer and then apply Linear,ReLU,Linear and Sigmoid activations.

Lets use **SENet** to predict CIFAR-10 dataset.

In [9]:
#Download the dataset and transform them to the format for ResNet
train_transform = transforms.Compose([
        transforms.Resize(224),             # resize shortest side to 224 pixels
        transforms.CenterCrop(224),         # crop longest side to 224 pixels at center
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])

test_transform = transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])

train_data = datasets.CIFAR10(root='./', train=True, download=True, transform=train_transform)
test_data = datasets.CIFAR10(root='./', train=False, download=True, transform=test_transform)

Files already downloaded and verified
Files already downloaded and verified


In [11]:
#Create dataloaders
torch.manual_seed(101)  # for reproducible results

train_loader = DataLoader(train_data, batch_size=10, shuffle=True)
test_loader = DataLoader(test_data, batch_size=10, shuffle=False)

In [13]:
def count_parameters(model):
    params = [p.numel() for p in model.parameters() if p.requires_grad]
    for item in params:
        print(f'{item:>6}')
    print(f'______\n{sum(params):>6}')

def train(model, criterion, optimizer):
    #Lets train the model
    import time
    start_time = time.time()

    epochs = 1

    max_trn_batch = 800
    max_tst_batch = 300

    train_losses = []
    test_losses = []
    train_correct = []
    test_correct = []

    for i in range(epochs):
        trn_corr = 0
        tst_corr = 0
    
        # Run the training batches
        for b, (X_train, y_train) in enumerate(train_loader):
            if b == max_trn_batch:
                break
            b+=1
        
            # Apply the model
            y_pred = model(X_train)
            loss = criterion(y_pred, y_train)
 
            # Tally the number of correct predictions
            predicted = torch.max(y_pred.data, 1)[1]
            batch_corr = (predicted == y_train).sum()
            trn_corr += batch_corr
        
            # Update parameters
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Print interim results
            if b%200 == 0:
                print(f'epoch: {i:2}  batch: {b:4} [{10*b:6}/8000]  loss: {loss.item():10.8f}  \
accuracy: {trn_corr.item()*100/(10*b):7.3f}%')

        train_losses.append(loss)
        train_correct.append(trn_corr)

        # Run the testing batches
        with torch.no_grad():
            for b, (X_test, y_test) in enumerate(test_loader):
                if b == max_tst_batch:
                    break

                # Apply the model
                y_val = model(X_test)

                # Tally the number of correct predictions
                predicted = torch.max(y_val.data, 1)[1] 
                tst_corr += (predicted == y_test).sum()

        loss = criterion(y_val, y_test)
        test_losses.append(loss)
        test_correct.append(tst_corr)
        return test_correct

    print(f'\nDuration: {time.time() - start_time:.0f} seconds') # print the time elapsed

def analyze(model, name, linear_in):
    print('-'*100)
    print('-'*100)
    print("Starting", name)
    print(model)
    #count parameters
    print('-'*100)
    print("Parameters in full model")
    count_parameters(model)
    print('-'*100)
    #reset grad
    for param in model.parameters():
        param.requires_grad = False
    torch.manual_seed(42)
    model.fc = nn.Sequential(nn.Linear(linear_in, out_features=10, bias=True),
                                 nn.LogSoftmax(dim=1))
    print("Parameters to optimize")
    count_parameters(model)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.fc.parameters(), lr=0.001)
    print("Training")
    test_correct = train(model, criterion, optimizer)
    print(test_correct)
    print(f'Test accuracy: {test_correct[-1].item()*100/3000:.3f}%')
    print('-'*100)
    print('-'*100)
    print('\n\n')

In [14]:
#Lets update senet parameters for CIFAR dataset
analyze(senet, "senet", 2048)

----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Starting senet
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): SEBottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)


epoch:  0  batch:  200 [  2000/8000]  loss: 1.04844213  accuracy:  47.800%
epoch:  0  batch:  400 [  4000/8000]  loss: 0.98030359  accuracy:  56.000%
epoch:  0  batch:  600 [  6000/8000]  loss: 0.85683125  accuracy:  58.550%
epoch:  0  batch:  800 [  8000/8000]  loss: 0.79299808  accuracy:  60.812%
[tensor(1915)]
Test accuracy: 63.833%
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------



